# Grid Search using 50% Human Data, then Evaluate on other half

## Read Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sklearn.model_selection import train_test_split
from scipy.stats import anderson_ksamp

# Get data source
data_dir = 'data_process/12-06/'

# Simulated data (full model and before parameter inference)
simulated_results = data_dir + '0827_simulation_result_100m_50ep.csv'
# Human data
human_duration_data = data_dir + 'human_data_rr_error_rate.csv'
human_error_rate_data = data_dir + 'human_data_rr_time_cost.csv'

# Read in as dataframe.
df_simulations = pd.read_csv(simulated_results)
human_duration_df = pd.read_csv(human_duration_data)
human_error_rate_df = pd.read_csv(human_error_rate_data)

params = {
    'init_delta_t':None,
    'init_sigma_position_memory':None,
    'weight_memory_decay':None,
    'spatial_dist_coeff':None,
}

# Control the conditions
# Metrics
layouts = ['L100']
metrics = ['steps', 'error']
# Pre-defined parameters
prior_work_init_sigma_position_memory = 4.5
empirical_init_delta_time = 1

# Get individual-level human data
human_duration_data = [human_duration_df[col].tolist() for col in human_duration_df.columns if col != "Participant"]
human_error_data = [human_error_rate_df[col].tolist() for col in human_error_rate_df.columns if col != "Participant"]

In [2]:
human_duration_data

[[5.988023952,
  1.41509434,
  2.631578947,
  2.189781002,
  4.195804196,
  6.41025641,
  1.515151515,
  1.19047619,
  1.449275362,
  2.547770701,
  1.052631579,
  0.510204082,
  1.25,
  0.0,
  0.546448087,
  0.0,
  2.962962963,
  0.0,
  2.491103203,
  2.93040293,
  2.95202952,
  2.830188679,
  1.342281879,
  0.64516129,
  2.02020202,
  2.631578947,
  3.418803419,
  0.0,
  0.0,
  0.0]]

In [3]:
human_error_data

[[1.5,
  1.375,
  1.375,
  1.625,
  2.125,
  0.857142857,
  1.625,
  1.625,
  1.5,
  1.75,
  1.875,
  1.0,
  1.625,
  1.625,
  1.5,
  2.125,
  2.125,
  2.375,
  1.0,
  1.125,
  1.25,
  1.625,
  1.375,
  1.25,
  2.125,
  1.25,
  1.625,
  1.5,
  0.875,
  1.125]]

## Halve the human data into two parts - training and evaluation

In [4]:
# Normalize a list
def normalize(lst):
    min_val = min(lst)
    max_val = max(lst)
    if min_val == max_val:  # Avoid division by zero
        return [0.5 for _ in lst]  # Return 0.5 (middle) if all values are same
    return [(x - min_val) / (max_val - min_val) for x in lst]

# Define the cost function
def compute_cost(sim_data, human_data):
    return sum(abs(sim - human) for sim, human in zip(sim_data, human_data))

def find_best_params(data, human_duration, human_error):
    # Filter the main dataframe according to the pre-defined parameters
    data = data[
        (data['init_sigma_position_memory'] == prior_work_init_sigma_position_memory) 
        & (data['init_delta_t'] == empirical_init_delta_time)
    ]
    
    data_before_params_inf = data.copy()
    
    unique_params = data.drop(columns=['layout', 'steps', 'error']).drop_duplicates()
    min_cost = float('inf')
    best_params = None

    for index, row in unique_params.iterrows():
        filtered_df = data[
#             (data['init_delta_t'] == row['init_delta_t']) &
#             (data['init_sigma_position_memory'] == row['init_sigma_position_memory']) &
            (data['weight_memory_decay'] == row['weight_memory_decay']) &
            (data['spatial_dist_coeff'] == row['spatial_dist_coeff'])
        ]
        
        sim_steps = [filtered_df[filtered_df['layout'] == layout]['steps'].values[0] for layout in layouts]
        sim_error = [filtered_df[filtered_df['layout'] == layout]['error'].values[0] for layout in layouts]
        
        cost_steps = compute_cost(normalize(sim_steps), normalize(human_duration))
        cost_error = compute_cost(normalize(sim_error), normalize(human_error))
        
        total_cost = cost_steps + cost_error

        if total_cost < min_cost:
            min_cost = total_cost
            best_params = row
    
    # Given the best parameters and teh dataframe, return the simulated steps (durations) and errors for the three layouts
    sim_steps_best = []
    sim_errors_best = []
    
    # Extract the simulated steps and error for each layout based on best_params
    for layout in layouts:
        filtered_df = data[
#             (data['init_delta_t'] == best_params['init_delta_t']) &
#             (data['init_sigma_position_memory'] == best_params['init_sigma_position_memory']) &
            (data['weight_memory_decay'] == best_params['weight_memory_decay']) &
            (data['spatial_dist_coeff'] == best_params['spatial_dist_coeff']) &
            (data['layout'] == layout)
        ]
        
        sim_steps_best.append(filtered_df['steps'].values[0])
        sim_errors_best.append(filtered_df['error'].values[0])

    return best_params, min_cost, sim_steps_best, sim_errors_best

## Grid Search

In [5]:
%%time

num_iterations = 500
all_costs = []
all_best_params = []

# Lists to store sim-to-real mapping ratios for each iteration
sim_to_real_ratios_duration = []
sim_to_real_ratios_error = []

# Initialize dictionaries for storing results
simulated_durations = {'L100': []}
simulated_errors = {'L100': []}
human_train_durations = {'L100': []}
human_train_errors = {'L100': []}
human_test_durations = {'L100': []}
human_test_errors = {'L100': []}
human_train_durations_details = {'L100': []}  # Collect detailed data in each iterations, not aggregated
human_train_errors_details = {'L100': []}
human_test_durations_details = {'L100': []}
human_test_errors_details = {'L100': []}

for i in range(num_iterations):
    # Number of participants
    num_participants = len(human_duration_data[0])

    # Generate list of indices based on participants
    indices = list(range(num_participants))

    # Split indices to ensure consistency
    train_indices, test_indices = train_test_split(indices, test_size=0.5, random_state=i)  
    # using i as the seed for reproducibility

    # Use these indices to split the human data consistently across participants
    def split_data_based_on_indices(data, train_indices, test_indices):
        train_set = [data[i] for i in train_indices]
        test_set = [data[i] for i in test_indices]
        return train_set, test_set

    # Split human duration data
    train_duration = [split_data_based_on_indices(condition, train_indices, test_indices) for condition in human_duration_data]
    train_duration = list(zip(*train_duration))[0]  # Extracting the training data

    test_duration = [split_data_based_on_indices(condition, train_indices, test_indices) for condition in human_duration_data]
    test_duration = list(zip(*test_duration))[1]  # Extracting the test data

    # Split human error data
    train_error = [split_data_based_on_indices(condition, train_indices, test_indices) for condition in human_error_data]
    train_error = list(zip(*train_error))[0]  # Extracting the training data

    test_error = [split_data_based_on_indices(condition, train_indices, test_indices) for condition in human_error_data]
    test_error = list(zip(*test_error))[1]  # Extracting the test data

    # Compute mean for the training and test sets
    mean_train_duration = [np.mean(data) for data in train_duration]
    mean_train_error = [np.mean(data) for data in train_error]
    mean_test_duration = [np.mean(data) for data in test_duration]
    mean_test_error = [np.mean(data) for data in test_error]

    # Using the function to find the best parameters
    best_params, _, sim_durations, sim_errors = find_best_params(df_simulations, mean_train_duration, mean_train_error)
    all_best_params.append(best_params)
    
    # Get sim to real mapping ratios for duration and error metrics respectively
    # Compute sim-to-real ratio for the current iteration
    sim_to_real_ratio_duration = sum(mean_train_duration) / sum(sim_durations)
    sim_to_real_ratio_error = sum(mean_train_error) / sum(sim_errors)
    
    # Store the computed ratios
    sim_to_real_ratios_duration.append(sim_to_real_ratio_duration)
    sim_to_real_ratios_error.append(sim_to_real_ratio_error)
    
    for label, sd, se, ted, tee, trd, tre in zip(
        ['L100'], 
        sim_durations, 
        sim_errors, 
        test_duration, 
        test_error, 
        train_duration, 
        train_error,
    ):
        simulated_durations[label].append(sd)
        simulated_errors[label].append(se)
        human_train_durations[label].append(np.mean(trd))
        human_train_errors[label].append(np.mean(tre))
        human_test_durations[label].append(np.mean(ted))
        human_test_errors[label].append(np.mean(tee))
        human_train_durations_details[label].append(trd)
        human_train_errors_details[label].append(tre)
        human_test_durations_details[label].append(ted)
        human_test_errors_details[label].append(tee)

CPU times: total: 49.4 s
Wall time: 51.4 s


In [6]:
# Copy results for reproducibility
import copy

simulated_durations_copy = copy.deepcopy(simulated_durations)
simulated_errors_copy = copy.deepcopy(simulated_errors)
human_test_durations_copy = copy.deepcopy(human_test_durations)
human_test_errors_copy = copy.deepcopy(human_test_errors)
human_train_durations_copy = copy.deepcopy(human_train_durations)
human_train_errors_copy = copy.deepcopy(human_train_errors)
human_test_durations_details_copy = copy.deepcopy(human_test_durations_details)
human_test_errors_details_copy = copy.deepcopy(human_test_errors_details)
human_train_durations_details_copy = copy.deepcopy(human_train_durations_details)
human_train_errors_details_copy = copy.deepcopy(human_train_errors_details)

In [7]:
import csv
# Open a CSV file in write mode
best_params_file_name = data_dir + "afterPI_rr_1204_1initdeltat_4dot5sigmapm_train_test_split_repeats_data_rep500.csv"

In [8]:
# Write
with open(best_params_file_name, "w", newline='') as csvfile:
    # Initialize the CSV writer
    writer = csv.writer(csvfile)

    # Write the headers
    headers = [
        "Layout", 
        "Simulated Durations", 
        "Simulated Errors", 
        "Simulated and Tuned Durations",
        "Simulated and Tuned Errors",
        "Human Train Durations", 
        "Human Train Errors", 
        "Human Test Durations",
        "Human Test Errors",
        "init_delta_t", 
        "init_sigma_position_memory", 
        "weight_memory_decay", 
        "spatial_dist_coeff",
        "Sim to Real Ratio Duration",
        "Sim to Real Ratio Error"
    ]
    writer.writerow(headers)
    
    # Write data for each layout
    for i, best_param in enumerate(all_best_params):
        for label in ['L100']:
            row = [
                label, 
                simulated_durations[label][i], 
                simulated_errors[label][i],
                simulated_durations[label][i] * sim_to_real_ratios_duration[i],
                simulated_errors[label][i] * sim_to_real_ratios_error[i],
                human_train_durations[label][i], 
                human_train_errors[label][i],
                human_test_durations[label][i],
                human_test_errors[label][i],
                best_param[0],  # init_delta_t
                best_param[1],  # init_sigma_position_memory
                best_param[2],  # weight_memory_decay
                best_param[3],  # spatial_dist_coeff
                sim_to_real_ratios_duration[i],
                sim_to_real_ratios_error[i]
            ]
            writer.writerow(row)

## Data evaluation

In [9]:
import pandas as pd
import copy

# best_params_file_name = "study3_readingresumption/1118_normal_oneinitdeltat_4dot5sigmapm_train_test_split_repeats_data_rep500.csv"

# Read in the CSV file
data = pd.read_csv(best_params_file_name)

# Create dictionaries to store data by layout and type
simulated_durations = {'L100': []}
simulated_errors = {'L100': []}
human_train_durations = {'L100': []}
human_train_errors = {'L100': []}
human_test_durations = {'L100': []}
human_test_errors = {'L100': []}
sim_to_real_ratios_duration = {'L100': []}
sim_to_real_ratios_error = {'L100': []}
# sim_to_real_ratios_duration = []
# sim_to_real_ratios_error = []

# Iterate through each row in the dataset
for index, row in data.iterrows():
    layout = row['Layout']
    simulated_durations[layout].append(row['Simulated Durations'])
    simulated_errors[layout].append(row['Simulated Errors'])
    human_train_durations[layout].append(row['Human Train Durations'])
    human_train_errors[layout].append(row['Human Train Errors'])
    human_test_durations[layout].append(row['Human Test Durations'])
    human_test_errors[layout].append(row['Human Test Errors'])
    sim_to_real_ratios_duration[layout].append(row['Sim to Real Ratio Duration'])
    sim_to_real_ratios_error[layout].append(row['Sim to Real Ratio Error'])

num_iterations = int(data.shape[0] / len(layouts))

In [10]:
import matplotlib.pyplot as plt
import numpy as np

simulated_durations_copy = copy.deepcopy(simulated_durations)
simulated_errors_copy = copy.deepcopy(simulated_errors)
human_test_durations_copy = copy.deepcopy(human_test_durations)
human_test_errors_copy = copy.deepcopy(human_test_errors)
human_train_durations_copy = copy.deepcopy(human_train_durations)
human_train_errors_copy = copy.deepcopy(human_train_errors)

labels = ['L100']

# Calculate SEM
def std(data):
    return np.std(data)

# Function to adjust y-limit
def adjust_ylim(ax, means, sems):
    y_max = max([mean + sem for mean, sem in zip(means, sems)])
    ax.set_ylim(0, y_max * 1.15)  # Adding a bit of margin to the top

# Using the average results for the test set
mean_human_test_duration = [np.mean(human_test_durations_copy[label]) for label in labels]
std_human_test_duration = [std(human_test_durations_copy[label]) for label in labels]
mean_human_test_error = [np.mean(human_test_errors_copy[label]) for label in labels]
std_human_test_error = [std(human_test_errors_copy[label]) for label in labels]

# Adjust the simulated data with ratios
for iteration in range(num_iterations):
    for label in labels:
        simulated_durations_copy[label][iteration] *= sim_to_real_ratios_duration[layout][iteration]
        simulated_errors_copy[label][iteration] *= sim_to_real_ratios_error[layout][iteration]

# Now recompute the mean and sem for these adjusted simulated results
mean_simulated_duration = [np.mean(simulated_durations_copy[label]) for label in labels]
std_simulated_duration = [std(simulated_durations_copy[label]) for label in labels]
mean_simulated_error = [np.mean(simulated_errors_copy[label]) for label in labels]
std_simulated_error = [std(simulated_errors_copy[label]) for label in labels]

#########################################################################################################

# # Update the plotting logic
# x = np.arange(len(labels))
# width = 0.2

# fig, ax = plt.subplots(2, 1, figsize=(6, 8))

# # Plot for Duration
# rects1 = ax[0].bar(x - width/2, mean_human_test_duration, width, label='Human Data', color='blue', yerr=std_human_test_duration, capsize=10)
# rects2 = ax[0].bar(x + width/2, mean_simulated_duration, width, label='Simulated Data', color='green', yerr=std_simulated_duration, capsize=10)

# # Plot for Error
# rects3 = ax[1].bar(x - width/2, mean_human_test_error, width, label='Human Data', color='blue', yerr=std_human_test_error, capsize=10)
# rects4 = ax[1].bar(x + width/2, mean_simulated_error, width, label='Simulated Data', color='green', yerr=std_simulated_error, capsize=10)

# # Annotating bars
# for i, (rect1, rect2) in enumerate(zip(rects1, rects2)):
#     height1 = rect1.get_height()
#     height2 = rect2.get_height()
#     std1 = std_human_test_duration[i]
#     std2 = std_simulated_duration[i]
#     ax[0].annotate(f"{height1:.2f}\n({std1:.2f})", 
#                    (rect1.get_x() + rect1.get_width() / 2., height1 + std1 + 0.05),
#                    ha='center', va='bottom')
#     ax[0].annotate(f"{height2:.2f}\n({std2:.2f})", 
#                    (rect2.get_x() + rect2.get_width() / 2., height2 + std2 + 0.05),
#                    ha='center', va='bottom')

# for i, (rect3, rect4) in enumerate(zip(rects3, rects4)):
#     height3 = rect3.get_height()
#     height4 = rect4.get_height()
#     std3 = std_human_test_error[i]
#     std4 = std_simulated_error[i]
#     ax[1].annotate(f"{height3:.2f}\n({std3:.2f})", 
#                    (rect3.get_x() + rect3.get_width() / 2., height3 + std3 + 0.05),
#                    ha='center', va='bottom')
#     ax[1].annotate(f"{height4:.2f}\n({std4:.2f})", 
#                    (rect4.get_x() + rect4.get_width() / 2., height4 + std4 + 0.05),
#                    ha='center', va='bottom')

# # Labels, title, and custom x-axis tick labels
# ax[0].set_ylabel('Reading Resumption Time (s)')
# ax[0].set_xticks(x)
# ax[0].set_xticklabels(labels)
# ax[0].legend(fontsize=15)
# adjust_ylim(ax[0], mean_human_test_duration + mean_simulated_duration, std_human_test_duration + std_simulated_duration)

# ax[1].set_ylabel('Reading Resumption Error Rate (%)')
# ax[1].set_xticks(x)
# ax[1].set_xticklabels(labels)
# ax[1].legend(fontsize=15)
# adjust_ylim(ax[1], mean_human_test_error + mean_simulated_error, std_human_test_error + std_simulated_error)

# fig.tight_layout()
# plt.show()

### Save plots

In [11]:
x = np.array([0.5])
width = 0.2

fig_duration, ax_duration = plt.subplots(figsize=(6, 4))
rects1 = ax_duration.bar(x - width/2, mean_human_test_duration, width, label='Human Data', color='blue', yerr=std_human_test_duration, capsize=10)
rects2 = ax_duration.bar(x + width/2, mean_simulated_duration, width, label='Simulated Data', color='green', yerr=std_simulated_duration, capsize=10)

# Annotations for Duration plot
for i, (rect1, rect2) in enumerate(zip(rects1, rects2)):
    height1 = rect1.get_height()
    height2 = rect2.get_height()
    std1 = std_human_test_duration[i]
    std2 = std_simulated_duration[i]
    ax_duration.annotate(f"{height1:.2f}\n({std1:.2f})", 
                   (rect1.get_x() + rect1.get_width() / 2., height1 + std1 + 0.05),
                   ha='center', va='bottom')
    ax_duration.annotate(f"{height2:.2f}\n({std2:.2f})", 
                   (rect2.get_x() + rect2.get_width() / 2., height2 + std2 + 0.05),
                   ha='center', va='bottom')

# Labels for Duration plot
ax_duration.set_ylabel('Reading Resumption Time (s)', fontsize=14)
ax_duration.set_xticks(x)
ax_duration.set_xticklabels(labels, fontsize=12, fontweight='bold')
ax_duration.tick_params(axis='y', labelsize=14)  # Adjust the fontsize as desired for y-axis

ax_duration.legend(fontsize=15)
adjust_ylim(ax_duration, mean_human_test_duration + mean_simulated_duration, std_human_test_duration + std_simulated_duration)

# Also, if you want the axis tick numbers to be larger:
ax_duration.tick_params(axis='both', labelsize=12)

# fig_duration.tight_layout()
fig_duration.savefig(data_dir + 'study4readingresumptiontimecost.png', dpi=300)
plt.close(fig_duration)

In [12]:
fig_error, ax_error = plt.subplots(figsize=(6, 4))
rects3 = ax_error.bar(x - width/2, mean_human_test_error, width, label='Human Data', color='blue', yerr=std_human_test_error, capsize=10)
rects4 = ax_error.bar(x + width/2, mean_simulated_error, width, label='Simulated Data', color='green', yerr=std_simulated_error, capsize=10)

# Annotations for Error plot
for i, (rect3, rect4) in enumerate(zip(rects3, rects4)):
    height3 = rect3.get_height()
    height4 = rect4.get_height()
    std3 = std_human_test_error[i]
    std4 = std_simulated_error[i]
    ax_error.annotate(f"{height3:.2f}\n({std3:.2f})", 
                   (rect3.get_x() + rect3.get_width() / 2., height3 + std3 + 0.05),
                   ha='center', va='bottom')
    ax_error.annotate(f"{height4:.2f}\n({std4:.2f})", 
                   (rect4.get_x() + rect4.get_width() / 2., height4 + std4 + 0.05),
                   ha='center', va='bottom')

# Labels for Error plot
ax_error.set_ylabel('Reading Resumption Error Rate (%)', fontsize=14)
ax_error.tick_params(axis='y', labelsize=14)  # Adjust the fontsize as desired for y-axis
ax_error.set_xticks(x)
ax_error.set_xticklabels(labels, fontsize=12, fontweight='bold')
ax_error.legend(fontsize=15)
adjust_ylim(ax_error, mean_human_test_error + mean_simulated_error, std_human_test_error + std_simulated_error)

# Also, if you want the axis tick numbers to be larger:
ax_duration.tick_params(axis='both', labelsize=12)

fig_error.tight_layout()

# fig_error.tight_layout()
fig_error.savefig(data_dir + 'study4readingresumptionerrorrate.png', dpi=300)
plt.close(fig_error)

### RSME

In [13]:
import pandas as pd

# Read in the CSV file
data = pd.read_csv(best_params_file_name)

# Create dictionaries to store data by layout and type
simulated_durations = {'L100': []}
simulated_errors = {'L100': []}
simulated_tuned_durations = {'L100': []}
simulated_tuned_errors = {'L100': []}
human_train_durations = {'L100': []}
human_train_errors = {'L100': []}
human_test_durations = {'L100': []}
human_test_errors = {'L100': []}
sim_to_real_ratios_duration = {'L100': []}
sim_to_real_ratios_error = {'L100': []}
# sim_to_real_ratios_duration = []
# sim_to_real_ratios_error = []

# Iterate through each row in the dataset
for index, row in data.iterrows():
    layout = row['Layout']
    simulated_durations[layout].append(row['Simulated Durations'])
    simulated_errors[layout].append(row['Simulated Errors'])
    simulated_tuned_durations[layout].append(row['Simulated and Tuned Durations'])
    simulated_tuned_errors[layout].append(row['Simulated and Tuned Errors'])
    human_train_durations[layout].append(row['Human Train Durations'])
    human_train_errors[layout].append(row['Human Train Errors'])
    human_test_durations[layout].append(row['Human Test Durations'])
    human_test_errors[layout].append(row['Human Test Errors'])
    sim_to_real_ratios_duration[layout].append(row['Sim to Real Ratio Duration'])
    sim_to_real_ratios_error[layout].append(row['Sim to Real Ratio Error'])

num_iterations = int(data.shape[0] / len(layouts))

In [20]:
# Get rsme
simulated_durations_copy = copy.deepcopy(simulated_durations)
simulated_errors_copy = copy.deepcopy(simulated_errors)
human_test_durations_copy = copy.deepcopy(human_test_durations)
human_test_errors_copy = copy.deepcopy(human_test_errors)
human_train_durations_copy = copy.deepcopy(human_train_durations)
human_train_errors_copy = copy.deepcopy(human_train_errors)
human_test_durations_details_copy = copy.deepcopy(human_test_durations_details)
human_test_errors_details_copy = copy.deepcopy(human_test_errors_details)
human_train_durations_details_copy = copy.deepcopy(human_train_durations_details)
human_train_errors_details_copy = copy.deepcopy(human_train_errors_details)


# Adjust the simulated data with ratios
for iteration in range(num_iterations):
    for label in labels:
        simulated_durations_copy[label][iteration] *= sim_to_real_ratios_duration[label][iteration]
        simulated_errors_copy[label][iteration] *= sim_to_real_ratios_error[label][iteration]

# RMSE computation function
def compute_rmse(true_vals, predicted_vals):
    return np.sqrt(np.mean((np.array(true_vals) - np.array(predicted_vals)) ** 2))

# Compute RMSE for durations and errors for each layout
rmse_durations_layouts = {}
rmse_errors_layouts = {}

for label in labels:
    rmse_durations_layouts[label] = compute_rmse(human_test_durations_copy[label], simulated_durations_copy[label])
    rmse_errors_layouts[label] = compute_rmse(human_test_errors_copy[label], simulated_errors_copy[label])

# Average RMSE across all layouts
avg_rmse_durations = np.mean(list(rmse_durations_layouts.values()))
avg_rmse_errors = np.mean(list(rmse_errors_layouts.values()))

# Display results
print(f"Average RMSE between simulated durations and human test durations: {avg_rmse_durations:.4f}")
print(f"Average RMSE between simulated errors and human test errors: {avg_rmse_errors:.4f}")
for label in labels:
    print(f"RMSE for {label} - Durations: {rmse_durations_layouts[label]:.4f}, Errors: {rmse_errors_layouts[label]:.4f}")
print(f"Average error rate is on scale: {np.mean(simulated_errors_copy['L100'])}")


Average RMSE between simulated durations and human test durations: 0.6288
Average RMSE between simulated errors and human test errors: 0.1373
RMSE for L100 - Durations: 0.6288, Errors: 0.1373
Average error rate is on scale: 1.5198404761858666


### Stastical Analysis: Anderson-Darling two-sample test

In [15]:
# Performing the Anderson-Darling two-sample test
error_rate_result = anderson_ksamp([human_test_errors_copy['L100'], simulated_errors_copy['L100']])
time_cost_result = anderson_ksamp([human_test_durations_copy['L100'], simulated_durations_copy['L100']])

C:\Users\91584\AppData\Local\Temp\ipykernel_49176\2323831652.py:3: UserWarning: p-value capped: true value larger than 0.25
  time_cost_result = anderson_ksamp([human_test_durations_copy['L100'], simulated_durations_copy['L100']])


In [16]:
print(f'Error Rate: {error_rate_result}, \nTime Cost: {time_cost_result}')

Error Rate: Anderson_ksampResult(statistic=3.0076130362371427, critical_values=array([0.325, 1.226, 1.961, 2.718, 3.752, 4.592, 6.546]), pvalue=0.01934134055579504), 
Time Cost: Anderson_ksampResult(statistic=-0.29283423182056, critical_values=array([0.325, 1.226, 1.961, 2.718, 3.752, 4.592, 6.546]), pvalue=0.25)
